## Загрузка библиотек и изучение данных

In [1]:
import pandas as pd
from pymystem3 import Mystem # загружаю библиотеку для лемматизации в будущем.
from collections import Counter # импортирую Counter для подсчета упоминаний в леммах слов

m = Mystem()
data = pd.read_csv('/datasets/data.csv')
display(data.head(10)) #Выводим первые 10 строк датасета, чтоб ознакомиться с данными.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.info() # Информация о файле, чтоб увидеть пропущенные значения и тип данных столбцов

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Судя по выводимым данным:
1. Общий трудовой стаж - в данных есть ошибки (отрицательных трудовой стаж, слишком огромные значения и пр.)
2. В образовании разным регистром написаны данные
3. Пустые данные в трудовом стаже и в зарплате соответственно

## Предобработка данных

In [3]:
data['days_employed'] = abs(data['days_employed']) # Приводим рабочий стаж к положительным значениям
print('Количество клиентов со стажем свыше 55 лет:', data[data['days_employed'] > 20000]['days_employed'].count()) # Считаем количество тех, у кого стаж выше 55 лет.

Количество клиентов со стажем свыше 55 лет: 3445


Шестая часть данных содержит информацию, что трудовой стаж слишком огромен. В целях нашей работы трудовой стаж не важен.

### Обработка пропусков

Пропущенные значения в трудовом стаже и зарплате могут быть по разной причине. Далеко не обязательно эти данные означают какую-то ошибку. Люди могут работать неофициально и получать доход, могут получать доход со сдачи квартир, в обход налогов, получать доход с акций, торговать киптовалютой и пр. А могут просто не работать, вступить в брак с содержательным человеком, вступить в наследство, вариантов много. Потому эти данные надо просто заполнить нулями.
(сначала я посчитал медианные значения по каждому типу занятости и думал по этому типу занятости присвоить значения во всех пропущенных данных)

In [4]:
print(data['total_income'].isna().sum() / len(data)) # Считаем количество пропусков в таблице

0.10099883855981417


Количество NaN в таблице составляет более 10% - это довольно много, чтоб пренебрегать этими данными.  
Однако если присваивать медианные значения, то количество заёмщиков по диапазонам заработка будет значительно выделяться в одной категории. Потому решено избавиться от незаполненных данных.

In [5]:
print(data['children'].unique()) # Уникальные значения столбца с количеством детей заёмщиков

[ 1  0  3  2 -1  4 20  5]


-1 ребенок значит, что детей нет и быть не может, или почему? Скорее всего тут должно быть значение 1. Как и 20 детей - это ошибка, должно быть 2. Проверим, насколько часто встречаются эти данные.

In [6]:
display(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Эти данные встречаются реже, чем 0,5%. Данными можно было бы и пренебречь, конечно, но я заменю их.

In [7]:
data['days_employed'] = data['days_employed'].fillna(value=0) # Меняем данные в столбце на 0
data['total_income'] = data['total_income'].fillna(value=0) # Меняем данные в столбце на 0
#data = data[data['children'] >= 0] # убираем отрицательные значения
data.loc[data['children'] == -1, 'children'] = 1
#data = data[data['children'] < 7] # убираем данные с 20 детьми.
data.loc[data['children'] == 20, 'children'] = 2
#print()
#print(data['children'].value_counts()) # здесь можно проверить, что все значения в столбце 'children' заменены.

Не меняя прежнего кода избавляемся от данных без указания дохода

In [8]:
data = data[data['total_income'] != 0] # убираю данные по кредиторам, доход которого был пропущен и заменен на 0.
data.info() # Проверяем наличие пропусков в данных.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null float64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null float64
purpose             19351 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 1.9+ MB


**Вывод**

Не обязательно отсутствующие значения - это серьезная ошибка в данных. Но, чтоб данные было возможно обработать - надо их заменить или удалить.

### Замена типа данных

Для обработки таблицы не нужны вещественные числа в зарплате и трудовом стаже клиентов.

In [9]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null int64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null int64
purpose             19351 non-null object
dtypes: int64(7), object(5)
memory usage: 1.9+ MB


**Вывод**

Сменить тип данных других столбцов не вижу смысла.

### Обработка дубликатов

Чтоб начать обработку дубликатов, сначала надо привести данные к нижнему регистру. Верхний встречается только в образовании и в столбце "пол". (есть там один персонаж, кстати, с неопределённым  полом)

In [10]:
data['education'] = data['education'].str.lower()
print(data.duplicated().sum(), 'дубликатов было обнаружно в данных') # Здесь я считаю количество дубликатов
data = data.drop_duplicates().reset_index(drop=True) # Удаляем дубликаты...
#data.info()

0 дубликатов было обнаружно в данных


**Вывод**

Дубликатов здесь было не так много, но от них всё равно лучше избавиться.

### Лемматизация

In [11]:
purpose_list = data['purpose'].unique() 
display(purpose_list) # Выведем все уникальные значения в столбце целей кредита, добавив в новую переменную.

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'получение образования',
       'сделка с подержанным автомобилем', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее образование',
       'покупка жилья для сдачи

In [12]:
lemmas = []
#Каждое значение в 'purpose_list' лемматизирую и добавляю в пустой список lemmas:
for a in purpose_list:
    lemm = m.lemmatize(a)
    lemmas.append(lemm)

Посмотрим, на какие категории делятся цели получения кредита, проверим, что чаще встречается в леммах.

In [13]:
lemm_text = []
for i in lemmas:
    lemm_text = list(lemm_text) + i
print(Counter(lemm_text))

Counter({' ': 59, '\n': 38, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


**Вывод**

По сути, все цели кредита делятся на 4 категории: автомобиль, образование, свадьба и недвижимость/жильё.

In [14]:
def purpose_func(purpose): # Создаю функцию, чтоб создать столбец 'purpose_id' и каждой категории присвоить id
    lemma = m.lemmatize(purpose)
    for i in lemma:
        if 'автомоб' in i: return 'автомобиль'
        if 'образ' in i: return 'образование'
        if 'свад' in i: return 'свадьба'
        if 'недвиж' in i: return 'недвижимость'
        if 'жил' in i: return 'недвижимость'
               
# Создаю столбец с целью кредита, после лемматицации.
data['purpose_id']= data['purpose'].apply(purpose_func)
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


In [15]:
print(data['purpose_id'].value_counts()) # Проверяю, какие цели кредита встречаются чаще всего.

недвижимость    9758
автомобиль      3897
образование     3597
свадьба         2099
Name: purpose_id, dtype: int64


Я лемматизировал данные для категоризации данных и объединения клиентов по цели получения займа

### Категоризация данных

Чтобы легче было просматривать данные, создаю отдельную таблицу, включающую количество детей, информацию по возврату кредита в срок, семейный статус, заработную плату и цель кредита

In [16]:
data_debt = data[['children','debt','family_status','total_income','purpose_id']] # Выделяю из data нужные столбцы
print(data_debt.head())

   children  debt     family_status  total_income    purpose_id
0         1     0   женат / замужем        253875  недвижимость
1         1     0   женат / замужем        112080    автомобиль
2         0     0   женат / замужем        145885  недвижимость
3         3     0   женат / замужем        267628   образование
4         0     0  гражданский брак        158616       свадьба


Выделил я эти столбцы как для облегчения визуализации, так и для быстродействия системы. Теперь можно проще работать с таблицей, например:

In [17]:
print('Количество заемщиков в каждой категории по количеству детей:')
display(data_debt['children'].value_counts()) 

Количество заемщиков в каждой категории по количеству детей:


0    12710
1     4387
2     1918
3      294
4       34
5        8
Name: children, dtype: int64

**Вывод**

Восприятие данных таблицы с меньшим количеством столбцов значительно лучше.

## Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Посмотрим таблицу с задолжностью заёмщиков и количеством детей.

In [18]:
#display(data_debt.groupby(['children','debt']).agg({'debt':'count'})) 
data_pivot_children = data_debt.pivot_table(index=['children'], columns='debt', values='purpose_id', aggfunc='count')
data_pivot_children = data_pivot_children.fillna(0) # убираю отсутствующие данные
# Считаю процент тех, у кого была задолжность по возврату кредита относительно общего количества заёмщиков
data_pivot_children['fraction'] = data_pivot_children[1] / (data_pivot_children[0]+data_pivot_children[1])

debt,0,1,fraction
children,,,
0,11758.0,952.0,0.074902
1,3978.0,409.0,0.093230
2,1733.0,185.0,0.096455
3,272.0,22.0,0.074830
4,31.0,3.0,0.088235
5,8.0,0.0,0.000000


**Вывод**

Наличие детей в семье увеличивает вероятность того, что кредит не вернут в срок на 0,6-2 %. У заёмщиков с 5 детьми трудно ориентироваться, очень мало статистических данных.

In [19]:
data_pivot_children.style.format({'fraction': '{:.2%}'})

debt,0,1,fraction
children,,,
0,11758,952,7.49%
1,3978,409,9.32%
2,1733,185,9.65%
3,272,22,7.48%
4,31,3,8.82%
5,8,0,0.00%


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создам таблицу с задолжностью заёмщиков и семейным положением. Также посчитаю долю тех, кто имел задолжность относительно общего числа.

In [20]:
data_pivot_family = data_debt.pivot_table(index=['family_status'], columns='debt', values='purpose_id', aggfunc='count')
data_pivot_family['fraction'] = data_pivot_family[1] / (data_pivot_family[0]+data_pivot_family[1])
data_pivot_family.style.format({'fraction': '{:.2%}'})

debt,0,1,fraction
family_status,,,
Не женат / не замужем,2271,254,10.06%
в разводе,1007,76,7.02%
вдовец / вдова,809,56,6.47%
гражданский брак,3396,339,9.08%
женат / замужем,10297,846,7.59%


**Вывод**

По этим данным можно сказать, что у тех, кто в браке, вдовец или в разводе - у тех категорий процент заёмщиков с возвратом кредита в срок значительно выше, чем у тех, кто не женат или в гражданском браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Разделение данных на категории заёмщиков по доходу. В них приблизительно по равному количеству заёмщиков.

In [21]:
def income_func(total): #Создаю функцию для расзделения заёмщиков по зарплатной категории.
    if total < 100000:            return 1
    if 100000 <= total <= 150000: return 2
    if 150000 < total < 200000:   return 3
    if 200000 <= total <= 300000: return 4
    if total > 300000:            return 5
data_debt['category_income'] = data_debt['total_income'].apply(income_func)
data_pivot_income = data_debt.pivot_table(index=['category_income'], columns='debt', values='purpose_id', aggfunc='count')
data_pivot_income['fraction'] = data_pivot_income[1] / (data_pivot_income[0]+data_pivot_income[1])
data_pivot_income.style.format({'fraction': '{:.2%}'})

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


debt,0,1,fraction
category_income,,,
1,4109,354,7.93%
2,5213,491,8.61%
3,3750,368,8.94%
4,3332,252,7.03%
5,1376,106,7.15%


**Вывод**

Зависимости между уровнем дохода и возвратом кредита в срок нет. Что заёмщики получают  менее 100.000, что больше 300.000 - разницы заметной нет.

По исследуемым данным можно сказать, что сильной зависимости между уровнем дохода и возвратом кредита нет. Однако заёмщики с доходом свыше 200 000 чуть более надёжные,скорее всего из-за своего дохода.

- Как разные цели кредита влияют на его возврат в срок?

In [22]:
data_pivot_purpose = data_debt.pivot_table(index=['purpose_id'], columns='debt', values='family_status', aggfunc='count')
data_pivot_purpose['fraction'] = data_pivot_purpose[1] / (data_pivot_purpose[0]+data_pivot_purpose[1])
data_pivot_purpose.style.format({'fraction': '{:.2%}'})

debt,0,1,fraction
purpose_id,,,
автомобиль,3530,367,9.42%
недвижимость,9043,715,7.33%
образование,3266,331,9.20%
свадьба,1941,158,7.53%


**Вывод**

Меньше всего возвращают кредит в срок те, кто берет заём на автомобиль или образование, заёмщики же на недвижимость - самые надёжные.

## Общий вывод

Было проверено несколько вопросов и найдены ответы на них, а именно:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
 - Наличие детей в семье увеличивает вероятность того, что кредит не вернут в срок на 0,6-2 %. 
 
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
 - У тех, кто в браке, вдовец или в разводе - у тех категорий процент заёмщиков с возвратом кредита в срок значительно выше, чем у тех, кто не женат или в гражданском браке.

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
 - Сильной зависимости между уровнем дохода и возвратом кредита нет. Однако заёмщики с доходом свыше 200 000 чуть более надёжные

4. Как разные цели кредита влияют на его возврат в срок?
 - Меньше всего возвращают кредит в срок те, кто берет заём на автомобиль или образование, заёмщики же на недвижимость - самые надёжные.